In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%matplotlib inline
 
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage.io import imread
from skimage.transform import resize
from glob import glob as g

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from tqdm.notebook import tqdm
from shutil import copy, move

In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray

target = []
flat_data = []
images = []
DataDirectory = '/kaggle/input/chss-pieces/chessPieces'
# Images to be classified as:
Categories = ["Knight","Queen","Rook", "Pawn", "King", "Bishop"]

for i in Categories:
  print("Category is:",i,"\tLabel encoded as:",Categories.index(i))
  
  target_class = Categories.index(i)
  
  path = os.path.join(DataDirectory,i)
 
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
   
    img_resized = resize(img_array,(150,150,3))
    flat_data.append(img_resized.flatten())
    images.append(img_resized)
    target.append(target_class)

flat_data = np.array(flat_data)
images = np.array(images)
target = np.array(target)

In [6]:
df = pd.DataFrame(flat_data)
df['Target'] = target
df

In [7]:
print(plt.imshow(images[0]))

In [8]:
from sklearn.model_selection import train_test_split

x = df.iloc[:,:-1].values
y = target
print("Input data dimensions:",x.shape)
print("Output data dimensions:",y.shape)

# Stratify ensures every image is divided in equal proportions (no bias)
x_train,x_test,y_train,y_test = train_test_split(x,y,shuffle=True,test_size = 0.2,random_state=101,stratify=y)
print("Dimensions of input training data:",x_train.shape)
print("Dimensions of input testing data:",x_test.shape)
print("Dimensions of output training data:",y_train.shape)
print("Dimensions of output testing data:",y_test.shape)


In [9]:
# Check if testing and training data are divided in equal proportions
print("Labels\t\t   Image index considered")
print(np.unique(y_train,return_counts=True))
print(np.unique(y_test,return_counts=True))

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]}]
                    
#GridSearch
cv = GridSearchCV(SVC(), tuned_parameters, refit = True,verbose= 3) 
cv.fit(x_train,y_train)

In [11]:
# Parameters: {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
print("Best parameters:",cv.best_params_)

svm = cv.best_estimator_
print("Model after tuning is:\n",svm)

In [12]:
y_prediction = svm.predict(x_test)

In [13]:
print("Expected results: ",y_test)
print("Predicted results:",y_prediction)

In [14]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

print("Confusion matrix results:\n",confusion_matrix(y_prediction,y_test))
print("\nClassification report of model:\n",classification_report(y_prediction,y_test))
print("Accuracy score:",100*accuracy_score(y_prediction,y_test))

In [15]:
import pickle
# Save SVM model in pickle file
pickle.dump(svm,open("Chess_model.p","wb"))

In [16]:
# Read byte from pickle model
test_model = pickle.load(open("Chess_model.p","rb"))

In [18]:
# Testing on new image
flat_data = []
url = input ('Enter url of image to test: ')
img_array = imread(url)

img_resized = resize(img_array,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data = np.array(flat_data)
print("Dimensions of original image are:",img_array.shape)
plt.imshow(img_resized)
y_output = test_model.predict(flat_data)
y_output = Categories[y_output[0]]
# URLs to test:
#../input/chss-pieces/chessPieces/Bishop/00000001.jpg
#../input/chss-pieces/chessPieces/Queen/00000003.jpg
print("Predicted:",y_output)